# Annihilation operator for a single boson

The cutoff L needs to be a power of 2 because we want to use qubit operators.

In [1]:
import numpy as np
from scipy.sparse import diags
L = 2  # cutoff for Fock space
a = diags(np.sqrt(np.linspace(1,L-1,L-1)),offsets=1)

## Representation in QISKIT

In [2]:
from qiskit.aqua.operators import MatrixOp

qubitOp = MatrixOp(primitive=a)
print(qubitOp.num_qubits)

1


Express the annihilation operator of a single boson in terms of N-qubit operators represented by Pauli matrices (2x2)

In [3]:
a_pauli = qubitOp.to_pauli_op()
print(a_pauli)

SummedOp([
  0.5 * X,
  0.5j * Y
])


In [4]:
a_pauli.to_matrix()

array([[0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j]])

# Identity operator for a single boson

In [5]:
from scipy.sparse import identity
iden = identity(L)

In [6]:
qubitOp = MatrixOp(primitive=iden)
print(qubitOp.num_qubits)

1


In [7]:
i_pauli = qubitOp.to_pauli_op()
print(i_pauli)

I


In [8]:
i_pauli.to_matrix()

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

# Create the Hamiltonian for 6 interacting bosons

Try to use the information in [this QISKIT tutorial](https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html)

In [9]:
Nmat = 6  # BMN2 for SU(2) has 6 bosonic matrices

In [10]:
from qiskit.aqua.operators.list_ops import ListOp, TensoredOp

boson = ListOp([i_pauli]*6)
print(boson)

ListOp([
  I,
  I,
  I,
  I,
  I,
  I
])


In [11]:
print(boson[0])

I


Since assignment is not possible, we construct the Nmat operators manually

In [12]:
# this does not work because the dimension of the spaces is not consisten
# boson0 = a_pauli^(i_pauli^5)

In [13]:
boson0 = TensoredOp([a_pauli, i_pauli, i_pauli, i_pauli, i_pauli, i_pauli])

In [14]:
print(boson0)

TensoredOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


In [15]:
boson0.num_qubits

6

In [16]:
boson0.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 1 stored elements in Compressed Sparse Row format>

In [17]:
boson0.to_matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])

In [18]:
boson0.to_matrix().shape

(64, 64)

In [19]:
boson1 = TensoredOp([i_pauli, a_pauli, i_pauli, i_pauli, i_pauli, i_pauli])

In [20]:
print(boson1)

TensoredOp([
  I,
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I
])


In [21]:
boson1.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 1 stored elements in Compressed Sparse Row format>

In [22]:
sparse = boson1.to_spmatrix()
type(sparse)

scipy.sparse.csr.csr_matrix

In [23]:
sparse.shape

(2, 2)

In [24]:
boson1.to_matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])

In [25]:
full = boson1.to_matrix()
len(full)

64

In [26]:
type(full)

numpy.ndarray

In [27]:
full.shape

(64, 64)

We can use a list of `ListOp` and then perform the tensor product using `TensoredOp`

In [28]:
# generically speaking, we construct the list of bosons and then take the outer product
a_list = []  # this will contain a1...a6 as a list of ListOp
for i in np.arange(0,Nmat):  # loop over all operators
    operator_list = [i_pauli] * Nmat  # only the identity repeated Nmat times
    operator_list[i] = a_pauli  # the i^th element is now the annihilation operator for a single boson
    a_list.append(ListOp(operator_list))


In [29]:
print(a_list[0])

ListOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


In [30]:
# here we create a list of operators which are the tensor products...
a_tensor = [TensoredOp(x) for x in a_list]


In [31]:
print(a_tensor[0])

TensoredOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


In [32]:
assert a_tensor[0] == boson0

In [33]:
print(a_tensor[0].to_pauli_op())

TensoredOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


The representation is already in terms of Pauli operators.

In [34]:
a_tensor[0].num_qubits

6

In [35]:
sparse = a_tensor[0].to_spmatrix()
type(sparse)

scipy.sparse.csr.csr_matrix

In [36]:
sparse.shape

(2, 2)

In [37]:
full = a_tensor[0].to_matrix()
len(full)

64

In [38]:
full.shape

(64, 64)

We also create the identity in this new tensor space of 6 bosons

In [39]:
i_tensor = TensoredOp(ListOp([i_pauli] * Nmat))

In [40]:
print(i_tensor)

TensoredOp([
  I,
  I,
  I,
  I,
  I,
  I
])


In [41]:
i_tensor.num_qubits

6

In [42]:
full = i_tensor.to_matrix()
len(full)

64

In [43]:
print(i_tensor.to_spmatrix())

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)


## Create the position operators

In [44]:
# for each boson they are constructed using a and adag
x_tensor = [1/np.sqrt(2)*(~x + x) for x in a_tensor]

In [45]:
print(x_tensor[0])

0.7071067811865475 * SummedOp([
  TensoredOp([
    SummedOp([
      0.5 * X,
      -0.5j * Y
    ]),
    I,
    I,
    I,
    I,
    I
  ]),
  TensoredOp([
    SummedOp([
      0.5 * X,
      0.5j * Y
    ]),
    I,
    I,
    I,
    I,
    I
  ])
])


## Create the full Hamiltonian

In [46]:
from qiskit.aqua.operators.list_ops import SummedOp

In [47]:
H_zero = 0.5*Nmat*i_tensor

In [48]:
print(H_zero)

3.0 * TensoredOp([
  I,
  I,
  I,
  I,
  I,
  I
])


In [49]:
H_zero.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

In [50]:
### Harmonic oscillator
# this should be summed over all the bosons (Nmat)
H_list = [H_zero]
for op in a_tensor:
    H_list.append((~op @ op))
H_osc = SummedOp(H_list)


In [51]:
print(H_osc)

SummedOp([
  3.0 * TensoredOp([
    I,
    I,
    I,
    I,
    I,
    I
  ]),
  ComposedOp([
    TensoredOp([
      SummedOp([
        0.5 * X,
        -0.5j * Y
      ]),
      I,
      I,
      I,
      I,
      I
    ]),
    TensoredOp([
      SummedOp([
        0.5 * X,
        0.5j * Y
      ]),
      I,
      I,
      I,
      I,
      I
    ])
  ]),
  ComposedOp([
    TensoredOp([
      I,
      SummedOp([
        0.5 * X,
        -0.5j * Y
      ]),
      I,
      I,
      I,
      I
    ]),
    TensoredOp([
      I,
      SummedOp([
        0.5 * X,
        0.5j * Y
      ]),
      I,
      I,
      I,
      I
    ])
  ]),
  ComposedOp([
    TensoredOp([
      I,
      I,
      SummedOp([
        0.5 * X,
        -0.5j * Y
      ]),
      I,
      I,
      I
    ]),
    TensoredOp([
      I,
      I,
      SummedOp([
        0.5 * X,
        0.5j * Y
      ]),
      I,
      I,
      I
    ])
  ]),
  ComposedOp([
    TensoredOp([
      I,
      I,
      I,
      SummedOp([
  

In [52]:
type(H_osc)

qiskit.aqua.operators.list_ops.summed_op.SummedOp

In [53]:
H_osc.to_pauli_op()

SummedOp([PauliOp(Pauli(z=[False, False, False, False, False, False], x=[False, False, False, False, False, False]), coeff=3.0), ComposedOp([TensoredOp([SummedOp([PauliOp(Pauli(z=[False], x=[True]), coeff=0.5), PauliOp(Pauli(z=[True], x=[True]), coeff=-0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0)], coeff=1.0, abelian=False), TensoredOp([SummedOp([PauliOp(Pauli(z=[False], x=[True]), coeff=0.5), PauliOp(Pauli(z=[True], x=[True]), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0)], coeff=1.0, abelian=False)], coeff=1.0, abelian=

In [54]:
H_osc.num_qubits

6

In [55]:
H_osc.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

In [56]:
quartic1 = SummedOp([x_tensor[2]@x_tensor[2]@x_tensor[3]@x_tensor[3], \
                    x_tensor[2]@x_tensor[2]@x_tensor[4]@x_tensor[4], \
                    x_tensor[1]@x_tensor[1]@x_tensor[3]@x_tensor[3], \
                    x_tensor[1]@x_tensor[1]@x_tensor[5]@x_tensor[5], \
                    x_tensor[0]@x_tensor[0]@x_tensor[4]@x_tensor[4], \
                    x_tensor[0]@x_tensor[0]@x_tensor[5]@x_tensor[5]])
                    

In [57]:
quartic2 = SummedOp([x_tensor[0]@x_tensor[2]@x_tensor[3]@x_tensor[5],\
                    x_tensor[0]@x_tensor[1]@x_tensor[3]@x_tensor[4],\
                    x_tensor[1]@x_tensor[2]@x_tensor[4]@x_tensor[5]], coeff = -2.0)

In [58]:
print(quartic2)

-2.0 * SummedOp([
  ComposedOp([
    0.7071067811865475 * SummedOp([
      TensoredOp([
        SummedOp([
          0.5 * X,
          -0.5j * Y
        ]),
        I,
        I,
        I,
        I,
        I
      ]),
      TensoredOp([
        SummedOp([
          0.5 * X,
          0.5j * Y
        ]),
        I,
        I,
        I,
        I,
        I
      ])
    ]),
    0.7071067811865475 * SummedOp([
      TensoredOp([
        I,
        I,
        SummedOp([
          0.5 * X,
          -0.5j * Y
        ]),
        I,
        I,
        I
      ]),
      TensoredOp([
        I,
        I,
        SummedOp([
          0.5 * X,
          0.5j * Y
        ]),
        I,
        I,
        I
      ])
    ]),
    0.7071067811865475 * SummedOp([
      TensoredOp([
        I,
        I,
        I,
        SummedOp([
          0.5 * X,
          -0.5j * Y
        ]),
        I,
        I
      ]),
      TensoredOp([
        I,
        I,
        I,
        SummedOp([
          0

In [59]:
type(quartic2)

qiskit.aqua.operators.list_ops.summed_op.SummedOp

In [60]:
### Quartic Interaction
V =  quartic1 + quartic2

In [61]:
V.num_qubits

6

In [62]:
V.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

## Define the 'tHooft coupling

In [63]:
g2 = 1.0

In [64]:
### Full Hamiltonian
H = H_osc + 0.5*g2*V

In [65]:
type(H)

qiskit.aqua.operators.list_ops.summed_op.SummedOp

In [66]:
H.to_pauli_op()

auliOp(Pauli(z=[True], x=[True]), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0)], coeff=0.7071067811865475, abelian=False), TensoredOp([PauliOp(Pauli(z=[False, False, False, False, False], x=[False, False, False, False, False]), coeff=1.0), SummedOp([PauliOp(Pauli(z=[False], x=[True]), coeff=0.5), PauliOp(Pauli(z=[True], x=[True]), coeff=-0.5j)], coeff=1.0, abelian=False)], coeff=0.7071067811865475, abelian=False)], coeff=-1.0, abelian=False), ComposedOp([TensoredOp([PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), SummedOp([PauliOp(Pauli(z=[False], x=[True]), coeff=0.5), PauliOp(Pauli(z=[True], x=[True]), coeff=-0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0), PauliOp(Pauli(z=[False], x=[False]), coeff=1.0)], coeff=0.7071067811865475, abelian=False), TensoredOp([PauliOp(Pauli(z=[False, False], x=[False, False]),

In [67]:
H.num_qubits

6

# Diagonalize the Hamiltonian to find the groundstate

In [68]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver

In [69]:
npme = NumPyMinimumEigensolver()
#result = npme.compute_minimum_eigenvalue(operator=H_osc)
result = npme.compute_minimum_eigenvalue(operator=H)

In [70]:
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.00000


I can not get anything different from 3.000 even if I have the interactions turned on...

## Construct the Hamiltonian matrix as usual
This only works for cutoffs smaller than 5 due to memory requirements in `MatrixOp`.

In [75]:
am = MatrixOp(a)

In [78]:
am.to_matrix()

array([[0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j]])